In [ ]:
%load_ext autoreload
%autoreload 2

%store -r the_page
%store -r the_editor
%store -r agg_actions
%store -r editor_inputname
%store -r calculator


if ('the_page' not in locals() or 
    'the_editor' not in locals() or 
    'agg_actions' not in locals() or 
    'editor_inputname' not in locals() or 
    'calculator' not in locals()):
    import pickle
    print("Loading default data...")
    the_page = pickle.load(open("data/the_page.p",'rb'))
    the_editor = pickle.load(open("data/the_editor.p",'rb'))
    agg_actions = pickle.load(open("data/agg_actions.p",'rb'))
    editor_inputname = pickle.load(open("data/editor_inputname.p",'rb'))
    calculator = pickle.load(open("data/calculator.p",'rb'))
    
from IPython.display import display, Markdown as md
display(md("---"))
display(md(f"# A. History of editor on a page"))
display(md(f"***Page: {the_page['title']}***"))
display(md(f"***Editor: {the_editor['name']}***"))

In [ ]:
editor_agg_actions = agg_actions[agg_actions['editor_id']==the_editor.userid]

#Listener
from visualization.actions_listener import ActionsListener
listener = ActionsListener(editor_agg_actions)
actions = (editor_agg_actions.loc[:,'total':'total_stopword_count'].columns.append(
    editor_agg_actions.loc[:,'adds':'reins_stopword_count'].columns)).values.tolist()

# Visualization
from utils.notebooks import get_date_slider_from_datetime
from ipywidgets import interact
from ipywidgets.widgets import Dropdown

interact(listener.listen, 
         _range = get_date_slider_from_datetime(agg_actions['year_month']),
         granularity=Dropdown(options=['Yearly', 'Monthly'], value='Monthly'),
         black=Dropdown(options=actions, value='total'), 
         red=Dropdown(options= ['None'] + actions, value='total_surv_48h'),
         green=Dropdown(options= ['None'] + actions, value='None'), 
         blue=Dropdown(options= ['None'] + actions, value='None'))
         

In [ ]:
from IPython.display import display, Markdown as md
display(md("---"))
display(md(f"# B. Token that enter into conflict with other editors"))
display(md(f"***Page: {the_page['title']}***"))
display(md(f"***Editor: {the_editor['name']}***"))

In [ ]:
editor_actions = calculator.actions[calculator.actions['editor']==editor_inputname]

# listener
from visualization.wordcloud_listener import WCListener
listener = WCListener(editor_actions)

# visualization
from utils.notebooks import get_date_slider_from_datetime
from ipywidgets import interact
from ipywidgets.widgets import Dropdown

interact(listener.listen, 
         _range=get_date_slider_from_datetime(editor_actions['rev_time']),
         action=Dropdown(options=['Both', 'Just Insertions', 'Just Deletions'], value='Both'),
         only_conflict = True)

In [ ]:
from IPython.display import display, Markdown as md
display(md("---"))
display(md(f"# C. Tokens in the page owner by the editor"))
display(md(f"***Page: {the_page['title']}***"))
display(md(f"***Editor: {the_editor['name']}***"))

In [ ]:
from visualization.owned_listener import OwnedListener
owned = calculator.get_all_tokens(stopwords=True)
listener = OwnedListener(owned, editor_inputname)

# Visualization
from utils.notebooks import get_date_slider_from_datetime
from ipywidgets import interact
from ipywidgets.widgets import Dropdown

interact(listener.listen,
         _range = get_date_slider_from_datetime(owned['rev_time']),
         granularity=Dropdown(options=['Yearly', 'Monthly', 'Daily'], value='Daily'),
         trace=Dropdown(options=['Tokens Owned (%)', 'Tokens Owned' ], value='Tokens Owned (%)'))

In [ ]:
from IPython.display import HTML
from utils.notebooks import get_next_notebook, get_previous_notebook

if len(editor_actions) > 0:
    display(HTML(f'<a href="{get_next_notebook()}" target="_blank">Go to next workbook</a>'))
else:
    display(HTML(f'<a href="{get_previous_notebook()}" target="_blank">Go back to the previous workbook</a>'))